In [7]:
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import Lossv2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(Lossv2.Loss(seperable=False,slim=False).to(device), (8, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 384, 256]           2,304
               ELU-2         [-1, 32, 384, 256]               0
            Conv2d-3         [-1, 32, 384, 256]           9,216
               ELU-4         [-1, 32, 384, 256]               0
            Conv2d-5         [-1, 32, 192, 128]           9,216
               ELU-6         [-1, 32, 192, 128]               0
            Conv2d-7         [-1, 32, 384, 256]           1,024
         MaxPool2d-8         [-1, 32, 192, 128]               0
               ELU-9         [-1, 32, 192, 128]               0
           Conv2d-10         [-1, 64, 192, 128]          25,088
           Conv2d-11         [-1, 64, 192, 128]          36,864
              ELU-12         [-1, 64, 192, 128]               0
           Conv2d-13           [-1, 64, 96, 64]          36,864
              ELU-14           [-1, 64,

In [1]:
import shutil
import torch
import torch.nn as nn
import numpy as np
torch.manual_seed(100)
np.random.seed(100)
import Lossv2
import generateLossImages
import time

global counter 
counter = 40

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def save_ckp(state, is_best, checkpoint_dir="./models/rest/", best_model_dir="./models/best/"):
    global counter 
    f_path = checkpoint_dir + str(counter) + '_checkpoint.pt'
    counter = counter + 1
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)

def load_ckp(model, optimizer, checkpoint_fpath="./models/rest/15_checkpoint.pt"):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch'], checkpoint['index'], checkpoint['min_lr'], checkpoint['max_lr'], checkpoint['steps'], checkpoint['step_size'], checkpoint['falling'], checkpoint['threshold']

In [2]:
# in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True

# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

loss_fn = nn.L1Loss(reduction='sum')
min_lr = 0.001
max_lr = 0.004
decay = 0.9
steps = 500
falling = True
start_epoch = 0
start_index = 0
momentum = 0.94
threshold = [0.16, 0.12, 0.09, 0.06, 0.04, 0.03, 0.02, 0.015, 0.01 -1]
step_size = (max_lr-min_lr)/steps
model = Lossv2.Loss(seperable=True, slim=True).to(device).to(memory_format=torch.channels_last)
#x = torch.rand(22, 8, 512, 768).to(device).to(memory_format=torch.channels_last)
model = torch.jit.script(model)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=momentum)
scaler = torch.cuda.amp.GradScaler(enabled=True)
#
#torch.autograd.set_detect_anomaly(False)


In [3]:
loss_fn = nn.L1Loss()
model = Lossv2.Loss(seperable=True, slim=True).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=momentum)
model, optimizer, start_epoch, start_index, min_lr, max_lr, steps, step_size, falling, threshold = load_ckp(model, optimizer)

In [5]:
#li = [31.007527351379395, 0.01501011848449707, 0.0010006427764892578, 28.98952603340149, 0.48702239990234375, 47.146605014801025, 12.32689380645752, 0.5665144920349121]
#su = sum(li)
#print([100*l/su for l in li])


In [4]:
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler

printing = True
running_loss = 0.
last_loss = 0.
lis = []
 # might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100
times = [0]*8
batch_size= 12

torch.backends.cudnn.benchmark = True
torch.autograd.detect_anomaly(check_nan=False)
torch.autograd.set_detect_anomaly(False, check_nan=False)
torch.autograd.profiler.profile(enabled=False)
torch.autograd.profiler.emit_nvtx(enabled=False)
#torch.autograd.gradgradcheck(check_undefined_grad= False)
#torch.autograd.gradcheck(check_undefined_grad= False)
#torch.no_grad()

scaler = GradScaler()
#import warnings 
#warnings.filterwarnings("ignore")

C:\Users\Rani\AppData\Local\Temp\ipykernel_4464\744408567.py:17: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  torch.autograd.detect_anomaly(check_nan=False)


In [4]:
torch.set_grad_enabled(True)
startup = False
for epoch in range(start_epoch, epochs):
    if startup:
        training = generateLossImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, startup = True)#, epoch=epoch) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
        #val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
        training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)#), worker_init_fn=worker_init_fn) # num_workers
        startup = True
        min_lr *= batch_size**0.5
        max_lr *= batch_size**0.5
        step_size = (max_lr-min_lr)/steps
        optimizer.param_groups[-1]['lr'] = max_lr
        los = [0]*(112//batch_size)
        pred = [0]*(112//batch_size)
        
    else:
        training = generateLossImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, epoch=epoch, startup = False) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
        #val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
        training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)#2)#), worker_init_fn=worker_init_fn) # num_workers
        #max_lr /= 10
        #min_lr /= 10
        #steps *= 2
        #step_size = (max_lr-min_lr)/steps
        #falling = True
        #optimizer.param_groups[-1]['lr'] = max_lr

    #end = 0
    #before_start = 0
    #in_upstart = True
    
    
    
    for index, data in enumerate(training_loader):#[90.06174230575562, 0.03202486038208008, 0.016014575958251953, 84.36967897415161, 1.3647308349609375, 137.4005196094513, 34.81299662590027, 1.9427943229675293, 0, 0]
        #print(len(inputs))
        #before_start += 1
        #if before_start > 55*11/batch_size and in_upstart:
        #    in_upstart = False
        #    times = [0]*8
        #start = time.time()
        #if end != 0:
        #    times[0] += (start-end)
        inputs, labels = data
        labels = torch.unsqueeze(labels, dim=-1)
        inputs.to(memory_format=torch.channels_last)
        #load = time.time()
        #times[1] += load-start

        #lab = time.time()
        #times[2] += (lab-load)

        # Make predictions for this batch
        with autocast():
            outputs = model(inputs)

            #out = time.time()
            #times[3] += (out-lab)

            # Compute the loss and its gradients
            
            loss = loss_fn(outputs, labels)

            #los = time.time()
            #times[4] += (los-out)

        loss.backward()#scaler.scale(loss).backward()#loss.backward()
        
        #back = time.time()
        #times[5] += (back-los)
        # Gather data and report
        if startup:
            los[index%(112//batch_size)] = loss.item()
            pred[index%(112//batch_size)] = (list(zip(labels.detach().cpu().numpy().tolist(),outputs.detach().cpu().numpy().tolist())))
            
        #runn = time.time()
        #times[6] += (runn-back)
        if startup:
            if index % (112//batch_size) == (112//batch_size)-1:
                optimizer.step()#scaler.step(optimizer)#optimizer.step()
                #scaler.update()
                optimizer.zero_grad(set_to_none=True)
        else:
            optimizer.step()#scaler.step(optimizer)#optimizer.step()
                #scaler.update()
            optimizer.zero_grad(set_to_none=True)
        if falling:
            optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
            if optimizer.param_groups[-1]['lr'] < min_lr:
                falling = False
                max_lr *= decay
                min_lr *= decay
                steps /= decay
                step_size = (max_lr-min_lr)/steps

                if printing:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!")
                    print(min_lr)
                    print(max_lr)
                    print(steps)
                    print(step_size)
                checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                save_ckp(checkpoint, True)
                
                if startup and sum(los)/10 < 0.05 and max(los) < 0.1:
                    #max_lr *= 0.4
                    #min_lr *= 0.4
                    #steps /= 0.4
                    #step_size = (max_lr-min_lr)/steps
                    #print("loss")
                    print(loss.item())
                    print("lr")
                    print(optimizer.param_groups[-1]['lr'])
                    print("pred")
                    print(labels)
                    print(outputs.T)
                    startup = False
                    max_lr /= 4
                    min_lr /= 4
                    steps *= 2
                    step_size = (max_lr-min_lr)/steps
                    falling = True
                    optimizer.param_groups[-1]['lr'] = max_lr
                    print("startup done !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    break


        else: 
            optimizer.param_groups[-1]['lr'] += step_size
            if optimizer.param_groups[-1]['lr'] > max_lr:
                falling = True

        if startup:
            if printing and index % (112//batch_size) == (112//batch_size)-1:
                print("loss")
                print(loss.item())
                print("lr")
                print(optimizer.param_groups[-1]['lr'])
                print("pred")
                print(pred)
                
        elif index % (20*(112//batch_size)-1) == 0:
        #    print("loss")
            print("loss")
            print(loss.item())
            print("lr")
            print(optimizer.param_groups[-1]['lr'])
            print("pred")
            print(labels.T)
            print(outputs.T)
        #        print('  batch {} loss: {}'.format(index + 1, last_loss))
        #        print(lis)
        #        lis = []
        #    running_loss = 0

        #    if last_loss < record:
        #        record = last_loss
        #        if last_loss < 0.08:
        #            checkpoint = {
        #            'epoch': epoch,
        #            'index': index,
        #            'min_lr': min_lr,
        #            'max_lr': max_lr,
        #            'steps': steps,
        #            'step_size': step_size,
        #            'falling': falling,
        #            'state_dict': model.state_dict(),
        #            'optimizer': optimizer.state_dict(),
        #            'threshold': threshold
        #        }
        #            save_ckp(checkpoint, True)
        
            
    #    end = time.time()
    #    times[7] += (end-runn)
        
    #    if before_start > 70*11/batch_size:
    #        break
    #break
    
        
                

KeyboardInterrupt: 

In [5]:
torch.set_grad_enabled(True)
startup = True
for epoch in range(start_epoch, epochs):

    if startup:
        training = generateLossImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, startup = True)
        training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)
        min_lr /= batch_size**0.5  # normally one puts the learning rate up by sqrt(batchsize), since that keeps the variance constant, however since we use sum in L1Loss, the oposit is true
        max_lr /= batch_size**0.5
        step_size = (max_lr-min_lr)/steps
        optimizer.param_groups[-1]['lr'] = max_lr
        los = [0]*(112//batch_size)
    else:
        training = generateLossImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, epoch=epoch, startup = False)
        training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)


    for index, data in enumerate(training_loader):
        inputs, labels = data
        labels = torch.unsqueeze(labels, dim=-1)
        inputs.to(memory_format=torch.channels_last)

        with autocast():
            outputs = model(inputs)            
            loss = loss_fn(outputs, labels)

        scaler.scale(loss).backward()#loss.backward()
        if startup:
            los[index%(112//batch_size)] = loss.item()/batch_size
            if index % (112//batch_size) == (112//batch_size)-1:         
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
        else:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        if falling:
            optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
            if optimizer.param_groups[-1]['lr'] < min_lr:
                falling = False
                max_lr *= decay
                min_lr *= decay
                steps /= decay
                step_size = (max_lr-min_lr)/steps
                if printing:
                    print("Saving model !!")
                    print("Min lr")
                    print(min_lr)
                    print("Max lr")
                    print(max_lr)
                checkpoint = {'epoch': epoch, 'index': index, 'min_lr': min_lr, 'max_lr': max_lr, 'steps': steps, 'step_size': step_size, 'falling': falling, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'threshold': threshold
                }
                save_ckp(checkpoint, True)
                
                if startup and sum(los)/10 < 0.05 and max(los) < 0.1:
                    startup = False
                    max_lr /= 4
                    min_lr /= 4
                    steps *= 2
                    step_size = (max_lr-min_lr)/steps
                    falling = True
                    optimizer.param_groups[-1]['lr'] = max_lr
                    print("startup done ! !")
                    break
        else: 
            optimizer.param_groups[-1]['lr'] += step_size
            if optimizer.param_groups[-1]['lr'] > max_lr:
                falling = True

        if (startup and printing and index % (112//batch_size) == (112//batch_size)-1) or (not startup and printing and index % (20*(112//batch_size) == (112//batch_size))-1):
            print("loss")
            print(loss.item())
            print("lr")
            print(optimizer.param_groups[-1]['lr'])
            print("pred")
            print(labels.T)
            print(outputs.T)

loss
2.045703172683716
lr
0.0011391120811111322
pred
tensor([[0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.8000, 0.8000,
         0.8000, 0.8000, 0.8000]], device='cuda:0')
tensor([[0.4924, 0.5010, 0.5098, 0.5142, 0.5103, 0.5044, 0.4973, 0.5435, 0.5381,
         0.5479, 0.5337, 0.5205]], device='cuda:0', dtype=torch.float16,
       grad_fn=<PermuteBackward0>)
loss
1.2075682878494263
lr
0.0011235236238430128
pred
tensor([[0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000]], device='cuda:0')
tensor([[0.4927, 0.4951, 0.4946, 0.4824, 0.4941, 0.5049, 0.5059, 0.5000, 0.5151,
         0.5063, 0.5117, 0.4954]], device='cuda:0', dtype=torch.float16,
       grad_fn=<PermuteBackward0>)
loss
2.2891602516174316
lr
0.0011079351665748934
pred
tensor([[0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000,
         0.4000, 0.4000, 0.8000]], device='cuda:0')
tensor([[0.5820, 0.6016, 0.6055, 0.5796, 0.5640, 0.5923, 0.630

KeyboardInterrupt: 

In [ ]:
print(times)
su = sum(times)
print([100*l/su for l in times])


[0.0040051937103271484, 0.8806514739990234, 0.0, 0.1621570587158203, 0.001001119613647461, 0.31227612495422363, 0.0, 0.024025678634643555]
[0.2893682198971097, 63.62552420967629, 0.0, 11.715563045892052, 0.07232913598118719, 22.561402251625466, 0.0, 1.735813136927891]


In [ ]:
print(su/15*6645*50*4/86400)

1.4193603606135756
